In [2]:
import numpy as np
import sympy as sp
from scipy.interpolate import CubicSpline

# Define the function and parameters
def f(x):
    return np.exp(x) + np.sin((x + 2)/2)

a, b = -1, 0
control_points = [-0.29, -0.42, -0.76]

# ===== Q1: Linear Spline (1st degree) =====
print("="*50)
print("Q1: LINEAR SPLINE INTERPOLATION")
print("="*50)

# Set 1: Equidistant nodes
print("\n--- Set 1: Equidistant nodes ---")
x_equidistant = [a + k*(b-a)/4 for k in range(5)]
print(f"Nodes: {x_equidistant}")

# Build linear spline manually
x_sym = sp.Symbol('x')
linear_polys_equidistant = []

for i in range(len(x_equidistant)-1):
    x0, x1 = x_equidistant[i], x_equidistant[i+1]
    y0, y1 = f(x0), f(x1)

    # Linear polynomial: y = y0 + (y1-y0)/(x1-x0) * (x-x0)
    poly = y0 + (y1-y0)/(x1-x0) * (x_sym - x0)
    linear_polys_equidistant.append(poly)

    print(f"Interval [{x0:.2f}, {x1:.2f}]: S(x) = {sp.simplify(poly)}")

# Set 2: Chebyshev-like nodes
print("\n--- Set 2: Chebyshev-like nodes ---")
x_chebyshev = [(b-a)/2 * np.cos((2*k-1)/np.pi) + (a+b)/2 for k in range(1, 6)]
x_chebyshev = sorted(x_chebyshev)
print(f"Nodes: {[f'{x:.4f}' for x in x_chebyshev]}")

linear_polys_chebyshev = []

for i in range(len(x_chebyshev)-1):
    x0, x1 = x_chebyshev[i], x_chebyshev[i+1]
    y0, y1 = f(x0), f(x1)

    poly = y0 + (y1-y0)/(x1-x0) * (x_sym - x0)
    linear_polys_chebyshev.append(poly)

    print(f"Interval [{x0:.4f}, {x1:.4f}]: S(x) = {sp.simplify(poly)}")

# ===== Q2: Cubic Spline (3rd degree) =====
print("\n" + "="*50)
print("Q2: CUBIC SPLINE INTERPOLATION")
print("="*50)

print("\n--- Set 1: Equidistant nodes (Natural Cubic Spline) ---")
# Using scipy for cubic spline (natural boundary conditions)
cs_equidistant = CubicSpline(x_equidistant, [f(x) for x in x_equidistant], bc_type='natural')

print("Cubic polynomials for equidistant nodes:")
for i in range(len(x_equidistant)-1):
    x0, x1 = x_equidistant[i], x_equidistant[i+1]
    coeffs = cs_equidistant.c[:, i]
    poly = coeffs[0] + coeffs[1]*(x_sym-x0) + coeffs[2]*(x_sym-x0)**2 + coeffs[3]*(x_sym-x0)**3
    print(f"Interval [{x0:.2f}, {x1:.2f}]: S(x) = {sp.simplify(poly)}")

print("\n--- Set 2: Chebyshev-like nodes (Natural Cubic Spline) ---")
cs_chebyshev = CubicSpline(x_chebyshev, [f(x) for x in x_chebyshev], bc_type='natural')

print("Cubic polynomials for Chebyshev nodes:")
for i in range(len(x_chebyshev)-1):
    x0, x1 = x_chebyshev[i], x_chebyshev[i+1]
    coeffs = cs_chebyshev.c[:, i]
    poly = coeffs[0] + coeffs[1]*(x_sym-x0) + coeffs[2]*(x_sym-x0)**2 + coeffs[3]*(x_sym-x0)**3
    print(f"Interval [{x0:.4f}, {x1:.4f}]: S(x) = {sp.simplify(poly)}")

# ===== Q3: Error Calculation =====
print("\n" + "="*50)
print("Q3: ERROR CALCULATION AT CONTROL POINTS")
print("="*50)

def find_spline_value(x, nodes, polys):
    """Find which interval x belongs to and evaluate the spline"""
    for i in range(len(nodes)-1):
        if nodes[i] <= x <= nodes[i+1]:
            return polys[i].subs(x_sym, x)
    return None

print("\n--- Linear Spline Errors ---")
print("Control Point | True Value | Linear(Equidistant) | Linear(Chebyshev) | Error(Equidistant) | Error(Chebyshev)")
print("-" * 95)

for cp in control_points:
    true_val = f(cp)

    # Linear spline values
    lin_eq_val = find_spline_value(cp, x_equidistant, linear_polys_equidistant)
    lin_ch_val = find_spline_value(cp, x_chebyshev, linear_polys_chebyshev)

    print(f"{cp:13.2f} | {true_val:10.6f} | {lin_eq_val:18.6f} | {lin_ch_val:17.6f} | "
          f"{abs(true_val-lin_eq_val):18.6f} | {abs(true_val-lin_ch_val):17.6f}")

print("\n--- Cubic Spline Errors ---")
print("Control Point | True Value | Cubic(Equidistant) | Cubic(Chebyshev) | Error(Equidistant) | Error(Chebyshev)")
print("-" * 90)

for cp in control_points:
    true_val = f(cp)

    # Cubic spline values
    cub_eq_val = cs_equidistant(cp)
    cub_ch_val = cs_chebyshev(cp)

    print(f"{cp:13.2f} | {true_val:10.6f} | {cub_eq_val:17.6f} | {cub_ch_val:16.6f} | "
          f"{abs(true_val-cub_eq_val):17.6f} | {abs(true_val-cub_ch_val):16.6f}")

# Additional analysis: Method of Moments implementation
print("\n" + "="*50)
print("METHOD OF MOMENTS IMPLEMENTATION")
print("="*50)

def cubic_spline_method_of_moments(x_nodes, y_values):
    """Implement cubic spline using method of moments"""
    n = len(x_nodes) - 1
    h = [x_nodes[i+1] - x_nodes[i] for i in range(n)]

    # Build tridiagonal system for second derivatives
    A = np.zeros((n+1, n+1))
    b = np.zeros(n+1)

    # Natural spline boundary conditions
    A[0, 0] = 1
    A[n, n] = 1

    # Internal equations
    for i in range(1, n):
        A[i, i-1] = h[i-1]
        A[i, i] = 2*(h[i-1] + h[i])
        A[i, i+1] = h[i]
        b[i] = 6*((y_values[i+1] - y_values[i])/h[i] - (y_values[i] - y_values[i-1])/h[i-1])

    # Solve for second derivatives
    M = np.linalg.solve(A, b)

    # Calculate coefficients
    coefficients = []
    for i in range(n):
        a_i = y_values[i]
        b_i = (y_values[i+1] - y_values[i])/h[i] - h[i]*(2*M[i] + M[i+1])/6
        c_i = M[i]/2
        d_i = (M[i+1] - M[i])/(6*h[i])
        coefficients.append((a_i, b_i, c_i, d_i))

    return coefficients

print("\n--- Method of Moments (Equidistant nodes) ---")
y_equidistant = [f(x) for x in x_equidistant]
coeffs_mom_eq = cubic_spline_method_of_moments(x_equidistant, y_equidistant)

for i, (a, b, c, d) in enumerate(coeffs_mom_eq):
    x0 = x_equidistant[i]
    poly_mom = a + b*(x_sym-x0) + c*(x_sym-x0)**2 + d*(x_sym-x0)**3
    print(f"Interval [{x_equidistant[i]:.2f}, {x_equidistant[i+1]:.2f}]: S(x) = {sp.simplify(poly_mom)}")

print("\n--- Method of Moments (Chebyshev nodes) ---")
y_chebyshev = [f(x) for x in x_chebyshev]
coeffs_mom_ch = cubic_spline_method_of_moments(x_chebyshev, y_chebyshev)

for i, (a, b, c, d) in enumerate(coeffs_mom_ch):
    x0 = x_chebyshev[i]
    poly_mom = a + b*(x_sym-x0) + c*(x_sym-x0)**2 + d*(x_sym-x0)**3
    print(f"Interval [{x_chebyshev[i]:.4f}, {x_chebyshev[i+1]:.4f}]: S(x) = {sp.simplify(poly_mom)}")

Q1: LINEAR SPLINE INTERPOLATION

--- Set 1: Equidistant nodes ---
Nodes: [-1.0, -0.75, -0.5, -0.25, 0.0]
Interval [-1.00, -0.75]: S(x) = 0.840635383623326*x + 1.68794036339897
Interval [-0.75, -0.50]: S(x) = 0.922822376217963*x + 1.74958060784495
Interval [-0.50, -0.25]: S(x) = 1.03269946228586*x + 1.8045191508789
Interval [-0.25, 0.00]: S(x) = 1.18050679800186*x + 1.8414709848079

--- Set 2: Chebyshev-like nodes ---
Nodes: ['-0.9810', '-0.8055', '-0.5104', '-0.2112', '-0.0251']
Interval [-0.9810, -0.8055]: S(x) = 0.835091763096568*x + 1.68190102609139
Interval [-0.8055, -0.5104]: S(x) = 0.911031781537011*x + 1.74307220435882
Interval [-0.5104, -0.2112]: S(x) = 1.04067520853106*x + 1.80923907306292
Interval [-0.2112, -0.0251]: S(x) = 1.18426455394095*x + 1.83956039164849

Q2: CUBIC SPLINE INTERPOLATION

--- Set 1: Equidistant nodes (Natural Cubic Spline) ---
Cubic polynomials for equidistant nodes:
Interval [-1.00, -0.75]: S(x) = 0.847304979775645*x**3 + 3.36574489651519*x**2 + 4.18957